# Matrix of compositions to calculate liquidus temperature
This code runs over a matrix of composition for Cr and Zr wuth Cu as remainder.
It fixes a temperature range and runs equilibrium calculations
The output are files with the equilibrium calculation for each composition combination and a summary file with the liqudius temperature for each calculated composition

## Libraries

In [28]:
from pycalphad import Database, equilibrium, variables as v
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pycalphad.plot.utils import phase_legend
import os

### Defining file paths for database

In [29]:
dbf_diretory = r'C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\1 - databases\Copper\\'
db_name = 'Cu-Cr-Zr-Ni_Test3.tdb'

# dbf_diretory = r'C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\1 - databases\Copper\\'
# dbf_diretory = r'C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\1 - databases\\'

### Load database

In [30]:
# dbf = Database(dbf_diretory + 'steel_database_fix.tdb') #Steel database 
# dbf = Database(dbf_diretory + 'Qiu_(2020)_calpha_101734_CuCrNi.tdb')
# dbf = Database(dbf_diretory + 'Cu-Cr-Zr-Ni_Test3.tdb')
dbf = Database(dbf_diretory + db_name)


### Determine phases

In [31]:
# phases = list(dbf.phases.keys())

phases = ['LIQUID', 'FCC_A1', 'BCC_A2', 'HCP_A3',
           'CU5ZR_C15B', 'CU9ZR2', 'CU51ZR14', 'CU8ZR3', 'CU10ZR7', 'CUZR', 'CUZR2', 'C14_LAVES', 'C15_LAVES', 'C36_LAVES'] #For CuCrZr

print("Available phases:", phases)

Available phases: ['LIQUID', 'FCC_A1', 'BCC_A2', 'HCP_A3', 'CU5ZR_C15B', 'CU9ZR2', 'CU51ZR14', 'CU8ZR3', 'CU10ZR7', 'CUZR', 'CUZR2', 'C14_LAVES', 'C15_LAVES', 'C36_LAVES']


### Composition grid, temperature range, output folder

In [32]:
# --- Composition grid (wt%) ---
cr_wt_grid = [0.45, 0.65, 0.85, 1.05, 1.15]  # Cr wt%
zr_wt_grid = [0.05, 0.10, 0.15, 0.20, 0.25]  # Zr wt%

# --- Temperature range (Kelvin) ---
T_min = 1200.15
T_max = 1773.15
T_step = 0.5

# --- Output directory ---
output_dir = r'C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\6 - Equilibrium phase compositions\Copper\\'
os.makedirs(output_dir, exist_ok=True)
# --- Summary storage ---
summary = []

### Loop over compositions

In [33]:
for cr_wt in cr_wt_grid:
    for zr_wt in zr_wt_grid:
        # Mass fractions dictionary (Cu is remainder automatically)
        mass_fracs = {v.W('CR'): cr_wt/100, v.W('ZR'): zr_wt/100}

        # Compose string for filenames
        cu_wt = 100 - cr_wt - zr_wt
        comp_str = f'Cu{cu_wt:.3f}Cr{cr_wt:.3f}Zr{zr_wt:.3f}'
        print(f'Running equilibrium for {comp_str}')

        # Get mole fractions automatically
        conds = v.get_mole_fractions(mass_fracs, 'CU', dbf)
        conds[v.T] = (T_min, T_max, T_step)
        conds[v.P] = 1e5
        conds[v.N] = 1

        # Run equilibrium
        eq = equilibrium(dbf, ['CU','CR','ZR','VA'], phases, conds)

        # Broadcast temperatures to NP shape
        T_arr = eq.T.broadcast_like(eq.NP).values - 273.15  # Celsius
        phase_arr = eq.Phase.values
        frac_arr = eq.NP.values

        # Unique phases in this calculation
        unique_phases = sorted(set(phase_arr.flatten()) - {''})

        # Prepare data dictionary
        data = {"Temperature (°C)": np.unique(T_arr)}

        for ph in unique_phases:
            fractions = []
            for T in data["Temperature (°C)"]:
                mask = (T_arr == T) & (phase_arr == ph)
                if np.any(mask):
                    fractions.append(frac_arr[mask].max())  # take max fraction at this T
                else:
                    fractions.append(0)
            data[ph] = fractions

        df = pd.DataFrame(data)
        df["Sum of Phases"] = df[unique_phases].sum(axis=1)

        # --- Save per-composition CSV ---
        csv_file = os.path.join(output_dir, f'Eq_{comp_str}.csv')
        with open(csv_file, 'w') as f:
            f.write(f'Database: {db_name}\n')
            f.write(f'Composition (wt%): CU={cu_wt:.3f}, CR={cr_wt:.3f}, ZR={zr_wt:.3f}\n')
            f.write(f'Temperature range (K): {T_min} - {T_max} step {T_step}\n')
        df.to_csv(csv_file, mode='a', index=False)


Running equilibrium for Cu99.500Cr0.450Zr0.050
Running equilibrium for Cu99.450Cr0.450Zr0.100
Running equilibrium for Cu99.400Cr0.450Zr0.150
Running equilibrium for Cu99.350Cr0.450Zr0.200
Running equilibrium for Cu99.300Cr0.450Zr0.250
Running equilibrium for Cu99.300Cr0.650Zr0.050
Running equilibrium for Cu99.250Cr0.650Zr0.100
Running equilibrium for Cu99.200Cr0.650Zr0.150
Running equilibrium for Cu99.150Cr0.650Zr0.200
Running equilibrium for Cu99.100Cr0.650Zr0.250
Running equilibrium for Cu99.100Cr0.850Zr0.050
Running equilibrium for Cu99.050Cr0.850Zr0.100
Running equilibrium for Cu99.000Cr0.850Zr0.150
Running equilibrium for Cu98.950Cr0.850Zr0.200
Running equilibrium for Cu98.900Cr0.850Zr0.250
Running equilibrium for Cu98.900Cr1.050Zr0.050
Running equilibrium for Cu98.850Cr1.050Zr0.100
Running equilibrium for Cu98.800Cr1.050Zr0.150
Running equilibrium for Cu98.750Cr1.050Zr0.200
Running equilibrium for Cu98.700Cr1.050Zr0.250
Running equilibrium for Cu98.800Cr1.150Zr0.050
Running equil

In [34]:
import os
import pandas as pd
import numpy as np

# --- Directory containing equilibrium CSV files ---
output_dir = r'C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\6 - Equilibrium phase compositions\Copper'

# --- Summary storage ---
summary = []

# Loop over all CSV files in the output directory
for csv_file in os.listdir(output_dir):
    if csv_file.endswith(".csv") and csv_file.startswith("Eq_"):
        path = os.path.join(output_dir, csv_file)

        # Skip the first 3 rows of header info
        df = pd.read_csv(path, skiprows=3)

        if 'LIQUID' not in df.columns:
            continue

        # Make sure temperatures are sorted ascending
        df_sorted = df.sort_values('Temperature (°C)', ascending=True).reset_index(drop=True)

        # Find where LIQUID transitions from 1 to <1
        liq_vals = df_sorted['LIQUID'].values
        temps = df_sorted['Temperature (°C)'].values

        liquidus_temp = None
        for i in range(1, len(liq_vals)):
            if liq_vals[i-1] == 1 and liq_vals[i] < 1:
                liquidus_temp = temps[i]
                break

        # Extract composition from filename
        comp_str = csv_file.replace("Eq_", "").replace(".csv", "")
        summary.append({
            "Composition": comp_str,
            "Liquidus Temperature (C)": liquidus_temp
        })

# Convert to DataFrame
summary_df = pd.DataFrame(summary)

# Save summary CSV in the same directory
summary_file = os.path.join(output_dir, "Liquidus_summary.csv")
summary_df.to_csv(summary_file, index=False)
print(f"Summary successfully saved to {summary_file}")


Summary successfully saved to C:\PythonCode\Environments\Calphad\pycalphad\pycalphad codes\6 - Equilibrium phase compositions\Copper\Liquidus_summary.csv
